

Sistema de recomendação de posts para a rede social Arena RPG




In [ ]:
#Imports
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Upload dos dados
from google.colab import files
files.upload()

Saving dados.csv to dados.csv


{'dados.csv': b'postId,author,title,createdAt,type,system,descriptionLength,hasImage,hasComments,commentsCount,hasLikes,likesCount\r\n0,VoidNoxian,Procura-se jogadores,2019-01-10T00:00:00.000Z,mesa,Tormenta20,150,1,1,20,1,30\r\n1,RPGmaster,D\xc3\xbavida sobre D&D5e,2020-05-20T12:00:00.000Z,duvida,D&D 5e,200,0,1,10,1,25\r\n2,Aventureiro123,Jogadores para Tormenta20,2021-03-01T10:30:00.000Z,jogadores,Tormenta20,100,0,1,5,1,15\r\n3,DungeonMestre,Off topic - M\xc3\xbasicas para jogar RPG,2022-01-05T15:45:00.000Z,off topic,outros,300,0,1,50,1,75\r\n4,ElfMage,Mesa para jogar D&D5e,2021-09-10T20:15:00.000Z,mesa,D&D 5e,150,1,1,30,1,40\r\n5,MagoNegro,Procura-se jogadores para campanha de Ordem Paranormal,2020-08-15T14:20:00.000Z,jogadores,Ordem Paranormal,200,0,1,15,1,20\r\n6,RangerLunar,Mesa para jogar com personagens n\xc3\xa3o convencionais,2022-02-01T08:00:00.000Z,mesa,D&D 5e,250,1,1,40,1,60\r\n7,LuigiFan,HomeBrew Mario Kart,2021-02-20T10:30:00.000Z,off topic,D&D 5e,250,1,1,10,1,15\r\n8,Mes

In [ ]:
#Armazenar informações
df = pd.read_csv('dados.csv', encoding="unicode_escape", error_bad_lines=False)

<ipython-input-74-1f41edd8666f>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('dados.csv', encoding="unicode_escape", error_bad_lines=False)


In [ ]:
#Exibir dados
df

,postId,author,title,createdAt,type,system,descriptionLength,hasImage,hasComments,commentsCount,hasLikes,likesCount
0,0,VoidNoxian,Procura-se jogadores,2019-01-10T00:00:00.000Z,mesa,Tormenta20,150,1,1,20,1,30
1,1,RPGmaster,DÃºvida sobre D&D5e,2020-05-20T12:00:00.000Z,duvida,D&D 5e,200,0,1,10,1,25
2,2,Aventureiro123,Jogadores para Tormenta20,2021-03-01T10:30:00.000Z,jogadores,Tormenta20,100,0,1,5,1,15
3,3,DungeonMestre,Off topic - MÃºsicas para jogar RPG,2022-01-05T15:45:00.000Z,off topic,outros,300,0,1,50,1,75
4,4,ElfMage,Mesa para jogar D&D5e,2021-09-10T20:15:00.000Z,mesa,D&D 5e,150,1,1,30,1,40
5,5,MagoNegro,Procura-se jogadores para campanha de Ordem Pa...,2020-08-15T14:20:00.000Z,jogadores,Ordem Paranormal,200,0,1,15,1,20
6,6,RangerLunar,Mesa para jogar com personagens nÃ£o convencio...,2022-02-01T08:00:00.000Z,mesa,D&D 5e,250,1,1,40,1,60
7,7,LuigiFan,HomeBrew Mario Kart,2021-02-20T10:30:00.000Z,off topic,D&D 5e,250,1,1,10,1,15
8,8,MestreGame,Jogadores experientes para D&D5e,2022-09-10T14:45:00.000Z,jogadores,D&D5e,200,0,0,0,1,5
9,9,Galadriel,Aventura para iniciantes em Tormenta20,2020-05-05T18:20:00.000Z,mesa,Tormenta20,300,0,1,3,1,12


In [ ]:
#Combinar as caracteristicas
def combine_features(data):
  features = []
  for i in range(0, data.shape[0]):
    features.append(data['author'][i] + ' '+data['title'][i] + ' '+data['createdAt'][i] + ' '+data['type'][i] + ' '+ str(data['system'][i]) + ' '+str(data['descriptionLength'][i]) + ' '+ str(data['hasImage'][i]) + ' '+str(data['hasComments'][i]) + ' '+str(data['commentsCount'][i]) + ' '+str(data['hasLikes'][i]) + ' '+str(data['likesCount'][i]))
  
  return features

In [ ]:
#Criar coluna para armazenar as caracteristicas combinadas
df['combined_features']=combine_features(df)


In [ ]:
df

,postId,author,title,createdAt,type,system,descriptionLength,hasImage,hasComments,commentsCount,hasLikes,likesCount,combined_features
0,0,VoidNoxian,Procura-se jogadores,2019-01-10T00:00:00.000Z,mesa,Tormenta20,150,1,1,20,1,30,VoidNoxian Procura-se jogadores 2019-01-10T00:...
1,1,RPGmaster,DÃºvida sobre D&D5e,2020-05-20T12:00:00.000Z,duvida,D&D 5e,200,0,1,10,1,25,RPGmaster DÃºvida sobre D&D5e 2020-05-20T12:00...
2,2,Aventureiro123,Jogadores para Tormenta20,2021-03-01T10:30:00.000Z,jogadores,Tormenta20,100,0,1,5,1,15,Aventureiro123 Jogadores para Tormenta20 2021-...
3,3,DungeonMestre,Off topic - MÃºsicas para jogar RPG,2022-01-05T15:45:00.000Z,off topic,outros,300,0,1,50,1,75,DungeonMestre Off topic - MÃºsicas para jogar ...
4,4,ElfMage,Mesa para jogar D&D5e,2021-09-10T20:15:00.000Z,mesa,D&D 5e,150,1,1,30,1,40,ElfMage Mesa para jogar D&D5e 2021-09-10T20:15...
5,5,MagoNegro,Procura-se jogadores para campanha de Ordem Pa...,2020-08-15T14:20:00.000Z,jogadores,Ordem Paranormal,200,0,1,15,1,20,MagoNegro Procura-se jogadores para campanha d...
6,6,RangerLunar,Mesa para jogar com personagens nÃ£o convencio...,2022-02-01T08:00:00.000Z,mesa,D&D 5e,250,1,1,40,1,60,RangerLunar Mesa para jogar com personagens nÃ...
7,7,LuigiFan,HomeBrew Mario Kart,2021-02-20T10:30:00.000Z,off topic,D&D 5e,250,1,1,10,1,15,LuigiFan HomeBrew Mario Kart 2021-02-20T10:30:...
8,8,MestreGame,Jogadores experientes para D&D5e,2022-09-10T14:45:00.000Z,jogadores,D&D5e,200,0,0,0,1,5,MestreGame Jogadores experientes para D&D5e 20...
9,9,Galadriel,Aventura para iniciantes em Tormenta20,2020-05-05T18:20:00.000Z,mesa,Tormenta20,300,0,1,3,1,12,Galadriel Aventura para iniciantes em Tormenta...


In [ ]:
#Converter o texto da nova coluna para uma matriz
count_matrix = CountVectorizer().fit_transform(df['combined_features'])

In [ ]:
#Simlaridade entre vetores (cosine similarity)
cosine_similarity = cosine_similarity(count_matrix)

In [ ]:
#Printar os scores
print(cosine_similarity)

[[1.         0.29411765 0.45732956 0.20228869 0.40016337 0.40533961
  0.35400522 0.24253563 0.28583098 0.40016337 0.40016337 0.26136411
  0.43386092 0.24253563 0.24753689 0.50572174 0.32539569 0.46537892
  0.27022641 0.34299717 0.26462806 0.3796283  0.23338001]
 [0.29411765 1.         0.17149859 0.15171652 0.28583098 0.22518867
  0.30343304 0.30316953 0.34299717 0.28583098 0.28583098 0.21780342
  0.3796283  0.36380344 0.14852213 0.30343304 0.27116307 0.25854384
  0.22518867 0.28583098 0.21170245 0.3796283  0.14002801]
 [0.45732956 0.17149859 1.         0.14744196 0.33333333 0.35015049
  0.19658927 0.29462783 0.38888889 0.38888889 0.16666667 0.25400025
  0.31622777 0.29462783 0.4330127  0.44232587 0.26352314 0.45226702
  0.21884405 0.44444444 0.1028689  0.21081851 0.13608276]
 [0.20228869 0.15171652 0.14744196 1.         0.19658927 0.11616046
  0.26086957 0.31277162 0.24573659 0.19658927 0.44232587 0.11235088
  0.27975144 0.36490022 0.17025131 0.17391304 0.18650096 0.22227711
  0.193600

In [ ]:
#Pegar o titulo do post que o usuario interagiu
Title = df['title'][2]
Title

'Jogadores para Tormenta20'

In [ ]:
#Achar o postId do post que o usuario interagiu
post_id = df[df.title == Title]['postId'].values[0]
post_id

2

In [ ]:
#Pegar os scores de acordo com o post
scores = list(enumerate(cosine_similarity[post_id]))
print(scores)

[(0, 0.4573295603800236), (1, 0.17149858514250885), (2, 1.0000000000000002), (3, 0.14744195615489716), (4, 0.3333333333333334), (5, 0.3501504876259269), (6, 0.19658927487319622), (7, 0.2946278254943948), (8, 0.38888888888888895), (9, 0.38888888888888895), (10, 0.16666666666666669), (11, 0.254000254000381), (12, 0.31622776601683794), (13, 0.2946278254943948), (14, 0.4330127018922194), (15, 0.44232586846469146), (16, 0.26352313834736496), (17, 0.45226701686664544), (18, 0.2188440547662043), (19, 0.44444444444444453), (20, 0.10286889997472795), (21, 0.21081851067789198), (22, 0.13608276348795437)]


In [ ]:
#Ordenar a lista de scores em ordem descendente
sorted_scores = sorted(scores, key= lambda x:x[1], reverse = True)
#Remover o próprio post
sorted_scores = sorted_scores[1:]

In [ ]:
#Exibir os scores ordenados
sorted_scores

[(0, 0.4573295603800236),
 (17, 0.45226701686664544),
 (19, 0.44444444444444453),
 (15, 0.44232586846469146),
 (14, 0.4330127018922194),
 (8, 0.38888888888888895),
 (9, 0.38888888888888895),
 (5, 0.3501504876259269),
 (4, 0.3333333333333334),
 (12, 0.31622776601683794),
 (7, 0.2946278254943948),
 (13, 0.2946278254943948),
 (16, 0.26352313834736496),
 (11, 0.254000254000381),
 (18, 0.2188440547662043),
 (21, 0.21081851067789198),
 (6, 0.19658927487319622),
 (1, 0.17149858514250885),
 (10, 0.16666666666666669),
 (3, 0.14744195615489716),
 (22, 0.13608276348795437),
 (20, 0.10286889997472795)]

In [ ]:
#Sugerir 5 posts da lista ordenada
print(f'Sugestões de posts relacionados a "{Title}"')
i = 0
for post in sorted_scores:
  post_title = df[df.postId == post[0]]['title'].values[0]
  print(i+1, post_title)
  i = i+1
  if i >=5:
    break

Sugestões de posts relacionados a "Jogadores para Tormenta20"
1 Procura-se jogadores
2 Procura-se jogadores para campanha homebrew
3 Novos jogadores para D&D 5e
4 Mesa de Tormenta20 com vagas
5 Jogadores para campanha de Ordem Paranormal
